In [1]:
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
import trackpy
from iris.time import PartialDateTime

import tobac # #tobac package cloned from https://github.com/tobac-project/tobac.git

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [2]:
print(tobac.__version__)

1.5.0


In [3]:
# #load in file
# data_out=('../')
# data_file = '/data/users/hgilmour/olr/olr_1h/2005/olr_merge_jan_2005.nc' #this is the 1 hourly mean olr data for the first 20 days of 1998
# print(data_file)

# olr=iris.load_cube(data_file)


# #constraining the dataset by time so it runs quicker: 
# #week=iris.Constraint(time=lambda cell: cell.point.day<=7) #just the first 7 days
# #olr = olr.extract(week)

# olr.coord('time').bounds=None #REMOVING BOUNDS FROM TIME TO SEE IF THIS HELPS THE TYPEERROR
# # Remove coord system or else the animations don't run (suggested by AVD team)
# olr.coord('latitude').coord_system = None
# olr.coord('longitude').coord_system = None

# #code that AVD team sent:
# time = olr.coord('time')
# datetimes = time.units.num2date(time.points)
# con = iris.Constraint(time=datetimes[0])
# olr.extract(con)


# print(olr)

#Set up directory to save output and plots:
savedir=Path("Save")
if not savedir.is_dir():
    savedir.mkdir()
plot_dir=Path("Plot")
if not plot_dir.is_dir():
    plot_dir.mkdir()      


# #calculating dxy
# from math import pi
# longitude,latitude=np.meshgrid(olr.coord('longitude').points,olr.coord('latitude').points)
# R=6.3781e6
# dx=np.gradient(longitude)[1]
# dx=dx*(pi/180)*R*np.cos(latitude*pi/180)
# dy=np.gradient(latitude)[0]
# dy=dy*(pi/180)*R
# print(dx)
# print(dy)


# # Determine temporal and spatial sampling of the input data:
# dxy,dt=tobac.get_spacings(olr,grid_spacing=4500,time_spacing=3600) #time spacing = 1 hour

In [4]:
#dxy,dt=tobac.get_spacings(olr,grid_spacing=4500,time_spacing=3600) #time spacing = 1 hour


# #Convert from olr to Tb using Ohring and Gruber empirical formula:
# # # (1984) as given in Yang and Slingo (2001)
#     # Tf = tb(a+b*Tb) where a = 1.228 and b = -1.106e-3 K^-1
#     # OLR = sigma*Tf^4 
#     # where sigma = Stefan-Boltzmann constant = 5.67x10^-8 W m^-2 K^-4
# a = 1.228
# b = -1.106e-3
# sigma = 5.67e-8 # W m^-2 K^-4

# tf = (olr.data/5.67e-8)**0.25
# tb_var = (-1.228 + np.sqrt(1.228**2 + 4*-1.106e-3*tf.data))/(2*-1.106e-3)

# tb=olr.copy()
# tb.data=tb_var.data
# print(tb)
# iris.save(tb, '/data/users/hgilmour/olr/olr_1h/2005/tb_jan_2005.nc')

In [5]:
tb = iris.load_cube('/data/users/hgilmour/olr/olr_1h/2005/tb_jan_2005.nc')

In [6]:
print(tb)

toa_outgoing_longwave_flux / (W m-2) (time: 744; latitude: 1360; longitude: 1360)
    Dimension coordinates:
        time                              x              -                -
        latitude                          -              x                -
        longitude                         -              -                x
    Auxiliary coordinates:
        forecast_period                   x              -                -
    Scalar coordinates:
        forecast_reference_time      2002-01-01 00:00:00
    Cell methods:
        mean                         time (1 hour)
    Attributes:
        Conventions                  'CF-1.7'
        STASH                        m01s03i332
        source                       'Data from Met Office Unified Model'
        um_version                   '10.6'


In [7]:
#calculating dxy
from math import pi
longitude,latitude=np.meshgrid(tb.coord('longitude').points,tb.coord('latitude').points)
R=6.3781e6
dx=np.gradient(longitude)[1]
dx=dx*(pi/180)*R*np.cos(latitude*pi/180)
dy=np.gradient(latitude)[0]
dy=dy*(pi/180)*R
print(dx)
print(dy)


# Determine temporal and spatial sampling of the input data:
dxy,dt=tobac.get_spacings(tb,grid_spacing=4500,time_spacing=3600) #time spacing = 1 hour

[[3451.3477 3451.3477 3451.3477 ... 3451.3477 3451.3477 3451.3477]
 [3453.3967 3453.3967 3453.3967 ... 3453.3967 3453.3967 3453.3967]
 [3455.4443 3455.4443 3455.4443 ... 3455.4443 3455.4443 3455.4443]
 ...
 [4356.1074 4356.1074 4356.1074 ... 4356.1074 4356.1074 4356.1074]
 [4355.286  4355.286  4355.286  ... 4355.286  4355.286  4355.286 ]
 [4354.4624 4354.4624 4354.4624 ... 4354.4624 4354.4624 4354.4624]]
[[4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.272  4508.272  4508.272  ... 4508.272  4508.272  4508.272 ]
 ...
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]
 [4508.4844 4508.4844 4508.4844 ... 4508.4844 4508.4844 4508.4844]]


In [8]:
tb.data.min()

179.43

In [9]:

#Parameter features:
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=0.5
parameters_features['target']='minimum'
parameters_features['threshold']=[240,50] #brightness temp threshold for anvil and cold core
parameters_features['n_min_threshold']=[1975,10] # number of grid points for 40,000km^2 area + smaller area for cold core

# Feature detection and save results to file:
print('starting feature detection')
Features=tobac.feature_detection_multithreshold(tb,dxy,**parameters_features, strict_thresholding=True)
Features.to_hdf('Save/cold_core/Features_240_50.h5','table')
print('feature detection performed and saved')

starting feature detection
feature detection performed and saved


In [10]:
Features

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,295.967957,295.967957,295.967957
1,0,328,680.480123,679.791400,1845983,50,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-12.480605,302.491613,302.491613,302.491613
2,1,7,179.948634,523.925062,4138,240,3,2005-01-01 01:30:00,2005-01-01 01:30:00,-32.752130,296.179031,296.179031,296.179031
3,1,289,680.619550,679.827687,1845462,50,4,2005-01-01 01:30:00,2005-01-01 01:30:00,-12.474959,302.493083,302.493083,302.493083
4,2,7,194.805129,505.538699,8143,240,5,2005-01-01 02:30:00,2005-01-01 02:30:00,-32.150444,295.434369,295.434369,295.434369
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,743,335,382.667845,1310.968198,283,50,2907,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.542002,328.054254,328.054254,328.054254
2907,743,336,384.800000,1260.600000,15,50,2908,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.455651,326.014349,326.014349,326.014349
2908,743,337,398.826087,1267.756522,115,50,2909,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.887594,326.304196,326.304196,326.304196
2909,743,339,413.516779,1291.348993,149,50,2910,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.292621,327.259678,327.259678,327.259678


In [11]:
# Segmentation:
parameters_segmentation={}
parameters_segmentation['target']='minimum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=240


# Perform segmentation and save results to files:
Mask_tb,Features_tb=tobac.segmentation_2D(Features,tb,dxy,**parameters_segmentation)
print('segmentation tb performed, start saving results to files')
iris.save([Mask_tb], 'Save/cold_core/Mask_Segmentation_tb_240_50.nc', zlib=True, complevel=4)
Features_tb.to_hdf('Save/cold_core/Features_tb_240_50.h5', 'table')
print('segmentation tb performed and saved')

segmentation tb performed, start saving results to files
segmentation tb performed and saved


In [12]:
# Linking:
parameters_linking={}
parameters_linking['v_max']=60 #(velocity of 60 m s-1 is referenced in https://journals.ametsoc.org/view/journals/mwre/126/6/1520-0493_1998_126_1630_lcvomc_2.0.co_2.xml#i1520-0493-126-6-1630-f01 study)
parameters_linking['stubs']=7 #minimum number of timesteps for a tracked cell to be reported (equivalent to 6 hours)
parameters_linking['order']=1
parameters_linking['extrapolate']=0 
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=15
parameters_linking['method_linking']= 'predict'



# Perform linking and save results to file:
Track=tobac.linking_trackpy(Features,tb,dt=dt,dxy=dxy,**parameters_linking)
Track["longitude"]=Track["longitude"]-360
Track.to_hdf('Save/cold_core/Track_240_50.h5','table')


Frame 743: 9 trajectories present.


In [13]:
df = pd.read_hdf('Save/cold_core/Track_240_50.h5','table')

In [14]:
df

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period,cell,time_cell
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,-64.032043,295.967957,295.967957,1,0 days 00:00:00
1,0,328,680.480123,679.791400,1845983,50,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-12.480605,-57.508387,302.491613,302.491613,2,0 days 00:00:00
2,1,7,179.948634,523.925062,4138,240,3,2005-01-01 01:30:00,2005-01-01 01:30:00,-32.752130,-63.820969,296.179031,296.179031,1,0 days 01:00:00
3,1,289,680.619550,679.827687,1845462,50,4,2005-01-01 01:30:00,2005-01-01 01:30:00,-12.474959,-57.506917,302.493083,302.493083,2,0 days 01:00:00
4,2,7,194.805129,505.538699,8143,240,5,2005-01-01 02:30:00,2005-01-01 02:30:00,-32.150444,-64.565631,295.434369,295.434369,1,0 days 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,743,335,382.667845,1310.968198,283,50,2907,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.542002,-31.945746,328.054254,328.054254,-1,NaT
2907,743,336,384.800000,1260.600000,15,50,2908,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.455651,-33.985651,326.014349,326.014349,-1,NaT
2908,743,337,398.826087,1267.756522,115,50,2909,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.887594,-33.695804,326.304196,326.304196,-1,NaT
2909,743,339,413.516779,1291.348993,149,50,2910,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.292621,-32.740322,327.259678,327.259678,-1,NaT


In [15]:
Track[Track.threshold_value == 50]

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,forecast_reference_time,forecast_period,cell,time_cell
1,0,328,680.480123,679.791400,1845983,50,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-12.480605,-57.508387,302.491613,302.491613,2,0 days 00:00:00
3,1,289,680.619550,679.827687,1845462,50,4,2005-01-01 01:30:00,2005-01-01 01:30:00,-12.474959,-57.506917,302.493083,302.493083,2,0 days 01:00:00
5,2,243,681.645727,680.251340,1841441,50,6,2005-01-01 02:30:00,2005-01-01 02:30:00,-12.433398,-57.489761,302.510239,302.510239,2,0 days 02:00:00
6,2,244,201.285714,495.642857,14,50,7,2005-01-01 02:30:00,2005-01-01 02:30:00,-31.887980,-64.966398,295.033602,295.033602,-1,NaT
8,3,195,681.760053,680.247407,1841024,50,9,2005-01-01 03:30:00,2005-01-01 03:30:00,-12.428768,-57.489920,302.510080,302.510080,2,0 days 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,743,335,382.667845,1310.968198,283,50,2907,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.542002,-31.945746,328.054254,328.054254,-1,NaT
2907,743,336,384.800000,1260.600000,15,50,2908,2005-01-31 23:30:00,2005-01-31 23:30:00,-24.455651,-33.985651,326.014349,326.014349,-1,NaT
2908,743,337,398.826087,1267.756522,115,50,2909,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.887594,-33.695804,326.304196,326.304196,-1,NaT
2909,743,339,413.516779,1291.348993,149,50,2910,2005-01-31 23:30:00,2005-01-31 23:30:00,-23.292621,-32.740322,327.259678,327.259678,-1,NaT
